In [49]:
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


In [26]:
df = pd.read_csv('global_warming_tweets.csv',encoding = "utf-8-sig",engine='python',error_bad_lines=False)


In [29]:
set(df['existence'].values)

{'N', 'No', 'Y', 'Yes', nan}

In [30]:
def decide_existence(existence):
    if existence == 'Y' or existence == 'Yes':
        return 1
    elif existence == 'N' or existence == 'No':
        return 0
    else:
        return -1

In [31]:
df['existence'] = df['existence'].apply(decide_existence)

In [32]:
df

,tweet,existence,existence.confidence
0,Global warming report urges governments to act...,1,1.0000
1,Fighting poverty and global warming in Africa ...,1,1.0000
2,Carbon offsets: How a Vatican forest failed to...,1,0.8786
3,Carbon offsets: How a Vatican forest failed to...,1,1.0000
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,1,0.8087
5,RT @sejorg: RT @JaymiHeimbuch: Ocean Saltiness...,1,1.0000
6,Global warming evidence all around us|A messag...,1,1.0000
7,Migratory Birds' New Climate Change Strategy: ...,1,1.0000
8,Southern Africa: Competing for Limpopo Water: ...,1,1.0000
9,"Global warming to impact wheat, rice productio...",1,1.0000


In [33]:
df = df[df['existence'] != -1]

In [35]:
df.to_csv('global_warming_tweets.csv')

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'].values, df['existence'].values, test_size=0.2)

In [50]:
# converts a collection of text documents to a matrix of token counts
vectorizer = TfidfVectorizer(lowercase=True)

In [51]:
# Learn a vocabulary dictionary of all tokens in X_train (list of reviews)
vectorizer.fit_transform(X_train)

<3380x8992 sparse matrix of type '<class 'numpy.float64'>'
	with 56232 stored elements in Compressed Sparse Row format>

In [52]:
classifier = LogisticRegression()

In [53]:
# vectorizer.transform transforms X_train into a document term matrix
# this fits the linear regression model using least squares regression
classifier.fit(vectorizer.transform(X_train), y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [54]:
print("Score:", classifier.score(vectorizer.transform(X_test), y_test))

Score: 0.8035502958579882


In [55]:
classifier.predict(vectorizer.transform(["Climate change is real and we should all be very worried"]))

array([1])